In [1]:
import sys  
sys.path.insert(0, '/Users/pratikaher/DGL/GRecSy/')

In [2]:
from dgl.data.utils import get_download_dir
import os
import pandas as pd
import numpy as np
import dgl
GENRES_ML_100K =\
    ['unknown', 'Action', 'Adventure', 'Animation',
     'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
     'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
     'Thriller', 'War', 'Western']
GENRES_ML_1M = GENRES_ML_100K[1:]

/Users/pratikaher/DGL/graph-rec/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
download_dir = get_download_dir()
_dir = os.path.join(download_dir, "ml-1m", "ml-1m")

In [4]:
def _load_rating_info(file_path, sep):

        rating_info = pd.read_csv(
            file_path, sep=sep, header=None,
            names=['user_id', 'movie_id', 'rating', 'timestamp'],
            dtype={'user_id': np.int32, 'movie_id' : np.int32,
                   'ratings': np.float32, 'timestamp': np.int64}, engine='python')
        return rating_info

def _load_raw_user_info(file_path):

    user_info = pd.read_csv(file_path, sep='::', header=None,
                            names=['id', 'gender', 'age', 'occupation', 'zip_code'], engine='python')

    return user_info
    
def _load_raw_movie_info(file_path):

        GENRES = GENRES_ML_1M

        movie_info = pd.read_csv(file_path, sep='::', header=None,
                                    names=['id', 'title', 'genres'], encoding='iso-8859-1')
        genre_map = {ele: i for i, ele in enumerate(GENRES)}
        genre_map['Children\'s'] = genre_map['Children']
        genre_map['Childrens'] = genre_map['Children']
        movie_genres = np.zeros(shape=(movie_info.shape[0], len(GENRES)), dtype=np.float32)
        for i, genres in enumerate(movie_info['genres']):
            for ele in genres.split('|'):
                if ele in genre_map:
                    movie_genres[i, genre_map[ele]] = 1.0
                else:
                    print('genres not found, filled with unknown: {}'.format(genres))
                    movie_genres[i, genre_map['unknown']] = 1.0
        
        for idx, genre_name in enumerate(GENRES):
            assert idx == genre_map[genre_name]
            movie_info[genre_name] = movie_genres[:, idx]
        
        movie_info.drop(columns=["genres"])

        return movie_info

In [5]:
user_data = _load_raw_user_info(os.path.join(_dir, 'users.dat'))
movie_data = _load_raw_movie_info(os.path.join(_dir, 'movies.dat'))
ratings_data = _load_rating_info(os.path.join(_dir, 'ratings.dat'), '::')

/var/folders/7w/pm43w7t11438_6102vq5jc6c0000gn/T/ipykernel_33316/2533451727.py:21: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movie_info = pd.read_csv(file_path, sep='::', header=None,


In [6]:
user_data.shape

(6040, 5)

In [8]:
movie_data.head(3)

,id,title,genres,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Animation|Children's|Comedy,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [34]:
num_test = int(np.ceil(ratings_data.shape[0] * 0.1))
shuffled_idx = np.random.permutation(ratings_data.shape[0])
all_train_rating_data = ratings_data.iloc[shuffled_idx[num_test: ]]

In [36]:
all_train_rating_data.shape

(900188, 4)

In [75]:
def _drop_unseen_nodes(orign_info, cmp_col_name, reserved_ids_set, label):
    
    pd_rating_ids = pd.DataFrame(list(reserved_ids_set), columns=["id_graph"])
    data_info = orign_info.merge(pd_rating_ids, left_on=cmp_col_name, right_on='id_graph', how='outer')
#     print(data_info)
    data_info = data_info.dropna(subset=[cmp_col_name, 'id_graph'])
    data_info = data_info.drop(columns=["id_graph"])
    data_info = data_info.reset_index(drop=True)
    return data_info

In [126]:
movie_data = _drop_unseen_nodes(orign_info=movie_data, cmp_col_name="id",
                                        reserved_ids_set=set(ratings_data["movie_id"].values), label="movie")

In [129]:
movie_data.shape

(3706, 21)

In [82]:
user_data = _drop_unseen_nodes(orign_info=user_data, cmp_col_name="id",
                                        reserved_ids_set=set(ratings_data["user_id"].values), label="user")

In [104]:
global_user_id_map = {ele: i for i, ele in enumerate(user_data['id'])}
global_user_id_reverse_map = {i: ele for i, ele in enumerate(user_data['id'])}

global_movie_id_map = {ele: i for i, ele in enumerate(movie_data['id'])}
global_movie_id_reverse_map = {i: ele for i, ele in enumerate(movie_data['id'])}

In [108]:
movie_data.shape

(3883, 21)

In [116]:
user_data

,id,gender,age,occupation,zip_code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [114]:
import torch

In [124]:
hetero_graph.nodes['user'].data['node_ID'] = torch.tensor(user_data['id'].to_numpy()).squeeze()

In [127]:
hetero_graph.nodes['movie'].data['node_ID'] = torch.tensor(movie_data['id'].to_numpy()).squeeze()

DGLError: Expect number of features to match number of nodes (len(u)). Got 3706 and 3953 instead.

In [120]:
hetero_graph = dgl.heterograph({('user', 'rates', 'movie'): (ratings_data['user_id'].to_numpy(), ratings_data['movie_id'].to_numpy())})

In [122]:
isolated_nodes =  (hetero_graph.out_degrees(etype='rates') == 0).nonzero().squeeze(1)
hetero_graph = dgl.remove_nodes(hetero_graph, isolated_nodes, ntype='user')

In [123]:
hetero_graph

Graph(num_nodes={'movie': 3953, 'user': 6040},
      num_edges={('user', 'rates', 'movie'): 1000209},
      metagraph=[('user', 'movie', 'rates')])

In [86]:
print('Total user number = {}, movie number = {}'.format(len(global_user_id_map), len(global_movie_id_map)))


Total user number = 6040, movie number = 3706


In [89]:
def _generate_pair_value(rating_info):
        rating_pairs = (np.array([global_user_id_map[ele] for ele in rating_info["user_id"]],
                                 dtype=np.int64),
                        np.array([global_movie_id_map[ele] for ele in rating_info["movie_id"]],
                                 dtype=np.int64))
        rating_values = rating_info["rating"].values.astype(np.float32)
        return rating_pairs, rating_values

In [100]:
def _generate_enc_graph(rating_pairs, add_support=False):

        data_dict = dict()
#         num_nodes_dict = {'user': self._num_user, 'movie': self._num_movie}
        rating_row, rating_col = rating_pairs

        data_dict.update({
                ('user', 'rates', 'movie'): (rating_row, rating_col),
                ('movie', 'rev-rates', 'user'): (rating_col, rating_row)
            })

        graph = dgl.heterograph(data_dict)

        # sanity check
        assert len(rating_pairs[0]) == sum([graph.number_of_edges(et) for et in graph.etypes]) // 2
 
        return graph

In [91]:
((x, y), z) = _generate_pair_value(ratings_data)

In [101]:
train_enc_graph = _generate_enc_graph((x, y))

In [103]:
train_enc_graph

Graph(num_nodes={'movie': 3706, 'user': 6040},
      num_edges={('movie', 'rev-rates', 'user'): 1000209, ('user', 'rates', 'movie'): 1000209},
      metagraph=[('movie', 'user', 'rev-rates'), ('user', 'movie', 'rates')])